In [1]:
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import glob
import librosa
import os
import time
from skimage.feature import hog, local_binary_pattern
from skimage import exposure
from skimage import feature
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
from sklearn.svm import SVC

train_path = "/kaggle/input/standard-ocr-dataset/data/training_data"
test_path = "/kaggle/input/standard-ocr-dataset/data/testing_data"

In [2]:
class_folders = [folder for folder in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, folder))]

classwise_imgs_paths = []
for image_dir in class_folders:
    p = [os.path.join(train_path, image_dir, filename) for filename in os.listdir(os.path.join(train_path, image_dir)) if filename.endswith(('.jpg', '.png'))]
    classwise_imgs_paths.append(p)

In [3]:
from scipy.ndimage import rotate
from skimage.util import random_noise
import cv2

def flip_image(image, axis=0):
    return np.flip(image, axis=axis)

def rotate_image(image, angle):
    return rotate(image, angle, reshape=False, mode='nearest')

def add_random_noise(image, noise_type='gaussian', seed=None):
    if noise_type == 'gaussian':
        noisy_image = random_noise(image, mode='gaussian', seed=seed)
    elif noise_type == 'salt':
        noisy_image = random_noise(image, mode='s&p', seed=seed, amount=0.04)
    elif noise_type == 'pepper':
        noisy_image = random_noise(image, mode='s&p', seed=seed, amount=0.04)
    elif noise_type == 's&p':
        noisy_image = random_noise(image, mode='s&p', seed=seed, amount=0.04)
    elif noise_type == 'speckle':
        noisy_image = random_noise(image, mode='speckle', seed=seed)
    else:
        noisy_image = image

    return np.clip(noisy_image, 0, 1)

def random_zoom(image, zoom_range=(0.5, 2.0)):
    zoom_factor = np.random.uniform(zoom_range[0], zoom_range[1])

    height, width = image.shape
    center_x, center_y = width // 2, height // 2
    transformation_matrix = cv2.getRotationMatrix2D((center_x, center_y), 0, zoom_factor)

    zoomed_image = cv2.warpAffine(image, transformation_matrix, (width, height))

    return zoomed_image

In [4]:
from skimage import io, color, filters
def compute_edges(image, sigma=1.0):
    
    edge_image = feature.canny(image, sigma=sigma)
    return edge_image

# s = compute_edges(np.array(Image.open("/kaggle/input/standard-ocr-dataset/data/testing_data/0/28310.png").convert('L')))
# plt.imshow(s)

In [5]:
data = []
data_edge = []
class_l = []
# data_normalised = []
for c in tqdm(classwise_imgs_paths, total=len(classwise_imgs_paths)):
    for image_p in c:
#         print(image_p)
        img = Image.open(image_p)
        img = img.convert('L')
        img = img.resize((38, 38), Image.Resampling.LANCZOS)
        img_arr = np.array(img).flatten()
        edge_arr = compute_edges(np.array(img))
        edge_arr_pil = Image.fromarray(edge_arr)
        edge_arr_pil = edge_arr_pil.resize((38, 38), Image.Resampling.LANCZOS)
        edge_arr = np.array(edge_arr_pil).flatten()
#         img_arr_norm = np.array(img).flatten() / 255.0
        cl = image_p.split("/")[-2]
        class_l.append(cl)
        data.append(img_arr)
        data_edge.append(edge_arr)
#         data_normalised.append(img_arr_norm)

  0%|          | 0/36 [00:00<?, ?it/s]

In [6]:
pixel_cols = [f'pixel_{i}' for i in range(np.array(data).shape[1])]
# 
edge_cols = [f'edge_{i}' for i in range(np.array(data_edge).shape[1])]
# 

data = np.array(data) / 255.0
data_edge = np.array(data_edge)
train_df = pd.DataFrame(data, columns=pixel_cols)
train_df_edge = pd.DataFrame(data_edge, columns=edge_cols)
# train_df_norm = pd.DataFrame(data_normalised, columns=pixel_cols)
labels_df = pd.DataFrame(class_l, columns=["label"])
train_df_w_labels_merged = pd.concat([train_df, labels_df], axis=1)

In [7]:
train_df_edge = train_df_edge.replace({True: 1, False: 0})
train_df_edge

,edge_0,edge_1,edge_2,edge_3,edge_4,edge_5,edge_6,edge_7,edge_8,edge_9,...,edge_1434,edge_1435,edge_1436,edge_1437,edge_1438,edge_1439,edge_1440,edge_1441,edge_1442,edge_1443
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20623,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20624,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20625,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20626,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
train_df_w_labels_merged

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_1435,pixel_1436,pixel_1437,pixel_1438,pixel_1439,pixel_1440,pixel_1441,pixel_1442,pixel_1443,label
0,0.862745,0.854902,0.858824,0.870588,0.870588,0.858824,0.866667,0.854902,0.850980,0.866667,...,0.862745,0.843137,0.800000,0.764706,0.764706,0.764706,0.788235,0.831373,0.843137,N
1,0.792157,0.784314,0.764706,0.745098,0.733333,0.737255,0.752941,0.752941,0.733333,0.737255,...,0.568627,0.556863,0.572549,0.631373,0.698039,0.764706,0.788235,0.792157,0.780392,N
2,0.862745,0.866667,0.878431,0.886275,0.874510,0.862745,0.862745,0.866667,0.874510,0.878431,...,0.870588,0.866667,0.858824,0.839216,0.796078,0.756863,0.741176,0.776471,0.811765,N
3,0.764706,0.764706,0.768627,0.764706,0.745098,0.737255,0.725490,0.729412,0.745098,0.760784,...,0.627451,0.588235,0.576471,0.600000,0.627451,0.690196,0.737255,0.756863,0.768627,N
4,0.819608,0.796078,0.780392,0.776471,0.768627,0.768627,0.756863,0.768627,0.772549,0.756863,...,0.580392,0.592157,0.580392,0.623529,0.713725,0.800000,0.815686,0.823529,0.815686,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20623,1.000000,1.000000,1.000000,0.996078,0.996078,0.996078,0.988235,0.992157,0.992157,0.996078,...,0.992157,0.988235,0.984314,0.984314,0.988235,0.992157,0.996078,1.000000,1.000000,J
20624,0.949020,0.956863,0.952941,0.937255,0.937255,0.941176,0.956863,0.964706,0.956863,0.960784,...,0.874510,0.909804,0.945098,0.956863,0.949020,0.956863,0.956863,0.956863,0.952941,J
20625,0.909804,0.909804,0.913725,0.917647,0.925490,0.925490,0.917647,0.913725,0.909804,0.913725,...,0.909804,0.921569,0.929412,0.925490,0.909804,0.909804,0.917647,0.917647,0.917647,J
20626,0.941176,0.945098,0.949020,0.952941,0.952941,0.956863,0.956863,0.949020,0.941176,0.941176,...,0.956863,0.956863,0.956863,0.956863,0.956863,0.952941,0.945098,0.945098,0.949020,J


In [12]:
def extract_hog_features(image):
    features, hog_image = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return features
hog_features = np.array([extract_hog_features(image.reshape(38, 38)) for image in train_df.values])
hog_features.shape

(20628, 324)

In [13]:
hog_cols = [f'hog_{i}' for i in range(hog_features.shape[1])] 
# 
hog_features_df = pd.DataFrame(hog_features, columns=hog_cols)

In [14]:
def extract_lbp_features(image):
    # Calculate LBP features
    radius = 1
    n_points = 8 * radius
    lbp_image = local_binary_pattern(image, n_points, radius, method='uniform')
    lbp_histogram, _ = np.histogram(lbp_image, bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_histogram = lbp_histogram / (lbp_histogram.sum() + 1e-6)
    return lbp_histogram

lbp_features = np.array([extract_lbp_features(image.reshape(38, 38)) for image in train_df.values])

/opt/conda/lib/python3.10/site-packages/skimage/feature/texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


In [15]:
lbp_cols = [f'lbp_{i}' for i in range(lbp_features.shape[1])]
lbp_features_df = pd.DataFrame(lbp_features, columns=lbp_cols)

In [16]:
train_df_norm_w_hog_features_and_lbp = pd.concat([train_df, hog_features_df, train_df_edge], axis=1)
train_df_norm_w_hog_features_and_lbp

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,edge_1434,edge_1435,edge_1436,edge_1437,edge_1438,edge_1439,edge_1440,edge_1441,edge_1442,edge_1443
0,0.862745,0.854902,0.858824,0.870588,0.870588,0.858824,0.866667,0.854902,0.850980,0.866667,...,0,0,0,0,0,0,0,0,0,0
1,0.792157,0.784314,0.764706,0.745098,0.733333,0.737255,0.752941,0.752941,0.733333,0.737255,...,0,0,0,0,0,0,0,0,0,0
2,0.862745,0.866667,0.878431,0.886275,0.874510,0.862745,0.862745,0.866667,0.874510,0.878431,...,0,0,0,0,0,0,0,0,0,0
3,0.764706,0.764706,0.768627,0.764706,0.745098,0.737255,0.725490,0.729412,0.745098,0.760784,...,0,0,0,0,0,0,0,0,0,0
4,0.819608,0.796078,0.780392,0.776471,0.768627,0.768627,0.756863,0.768627,0.772549,0.756863,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20623,1.000000,1.000000,1.000000,0.996078,0.996078,0.996078,0.988235,0.992157,0.992157,0.996078,...,0,0,0,0,0,0,0,0,0,0
20624,0.949020,0.956863,0.952941,0.937255,0.937255,0.941176,0.956863,0.964706,0.956863,0.960784,...,0,0,0,0,0,0,0,0,0,0
20625,0.909804,0.909804,0.913725,0.917647,0.925490,0.925490,0.917647,0.913725,0.909804,0.913725,...,0,0,0,0,0,0,0,0,0,0
20626,0.941176,0.945098,0.949020,0.952941,0.952941,0.956863,0.956863,0.949020,0.941176,0.941176,...,0,0,0,0,0,0,0,0,0,0


# Multi-Class LR**

In [17]:
alpha_map = {ele: i+10 for i, ele in enumerate(np.unique(labels_df.values)[10:])}
for i in range(0, 10):
    alpha_map[str(i)] = i
alpha_map

{'A': 10,
 'B': 11,
 'C': 12,
 'D': 13,
 'E': 14,
 'F': 15,
 'G': 16,
 'H': 17,
 'I': 18,
 'J': 19,
 'K': 20,
 'L': 21,
 'M': 22,
 'N': 23,
 'O': 24,
 'P': 25,
 'Q': 26,
 'R': 27,
 'S': 28,
 'T': 29,
 'U': 30,
 'V': 31,
 'W': 32,
 'X': 33,
 'Y': 34,
 'Z': 35,
 '0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9}

In [18]:
labels_df["label"] = labels_df["label"].map(alpha_map)

In [19]:
X = train_df_norm_w_hog_features_and_lbp.values
y = labels_df["label"].values

In [20]:
class MultiClassLogisticRegression:
    def __init__(self, input_dim, num_classes):
        self.W = np.random.rand(input_dim, num_classes)  # Initialize weights
        self.b = np.random.rand(num_classes)  # Initialize biases
        self.num_classes = num_classes

    def softmax(self, z):
        e_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return e_z / e_z.sum(axis=1, keepdims=True)

    def predict(self, X):
        z = np.dot(X, self.W) + self.b
        return self.softmax(z)

def categorical_cross_entropy_loss(y_true, y_pred):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    loss = -np.sum(y_true * np.log(y_pred)) / len(y_true)
    return loss

In [21]:
num_classes = 36
model = MultiClassLogisticRegression(X.shape[1], num_classes)
def one_hot_encode(labels, num_classes):
    num_samples = len(labels)
    encoded_labels = np.zeros((num_samples, num_classes))
    
    for i in range(num_samples):
        label = labels[i]
        encoded_labels[i, label] = 1
    
    return encoded_labels

y_enc = one_hot_encode(y, 36)

In [22]:
lr = 0.5
epochs = 5000
for epoch in range(epochs):
    predictions = model.predict(X)
    loss = categorical_cross_entropy_loss(y_enc, predictions)

    dW = np.dot(X.T, (predictions - y_enc)) / X.shape[0]
    db = np.sum(predictions - y_enc, axis=0) / X.shape[0]
    
    if loss < 0.3:
        lr = 0.15

    model.W -= lr * dW
    model.b -= lr * db

    if epoch%500==0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss}")

Epoch 1/5000, Loss: 16.272596482301996
Epoch 501/5000, Loss: 0.2467164953998887
Epoch 1001/5000, Loss: 0.18809551965153984
Epoch 1501/5000, Loss: 0.1503268709797448
Epoch 2001/5000, Loss: 0.12289942285808562
Epoch 2501/5000, Loss: 0.10207491745790145
Epoch 3001/5000, Loss: 0.08586010843711422
Epoch 3501/5000, Loss: 0.07305353992381652
Epoch 4001/5000, Loss: 0.06286024838122992
Epoch 4501/5000, Loss: 0.05470294022855063


In [23]:
lr = 0.1
epochs = 2000
for epoch in range(epochs):
    predictions = model.predict(X)
    loss = categorical_cross_entropy_loss(y_enc, predictions)

    dW = np.dot(X.T, (predictions - y_enc)) / X.shape[0]
    db = np.sum(predictions - y_enc, axis=0) / X.shape[0]
    
    model.W -= lr * dW
    model.b -= lr * db

    if epoch%500==0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss}")

Epoch 1/2000, Loss: 0.04815430200107741
Epoch 501/2000, Loss: 0.0445174221192316
Epoch 1001/2000, Loss: 0.041366331822093594
Epoch 1501/2000, Loss: 0.03862856759974456


In [24]:
lr = 0.05
epochs = 2000
for epoch in range(epochs):
    predictions = model.predict(X)
    loss = categorical_cross_entropy_loss(y_enc, predictions)

    dW = np.dot(X.T, (predictions - y_enc)) / X.shape[0]
    db = np.sum(predictions - y_enc, axis=0) / X.shape[0]
    
    model.W -= lr * dW
    model.b -= lr * db

    if epoch%500==0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss}")

Epoch 1/2000, Loss: 0.03624091072123749
Epoch 501/2000, Loss: 0.03516125367230841
Epoch 1001/2000, Loss: 0.03414969388731098
Epoch 1501/2000, Loss: 0.03320090017336846


In [25]:
# complete_trn_df = pd.concat([train_df, labels_df], axis=1)
# complete_trn_df

In [26]:
# def normalize_pixels(pixel_values):
#     return pixel_values / 255.0
# grouped = complete_trn_df.groupby('label')
# norm_test_data_map = {}

# for label, group in grouped:

#     df_normalized = group.copy()
    
#     pixel_intensity_columns = group.columns[:-1]  # Exclude the 'label' column
    
#     df_normalized[pixel_intensity_columns] = complete_trn_df[pixel_intensity_columns].apply(normalize_pixels)
#     df_normalized['label'] = group['label']
    
#     norm_test_data_map[label] = df_normalized
# training_df_ = pd.concat(norm_test_data_map.values(), ignore_index=True)

In [27]:
# training_df_

In [28]:
X_train, X_test, y_train, y_test = train_test_split(train_df_norm_w_hog_features_and_lbp, labels_df.values, test_size=0.2, random_state=42)

In [29]:
train_df_norm_w_hog_features_and_lbp

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,edge_1434,edge_1435,edge_1436,edge_1437,edge_1438,edge_1439,edge_1440,edge_1441,edge_1442,edge_1443
0,0.862745,0.854902,0.858824,0.870588,0.870588,0.858824,0.866667,0.854902,0.850980,0.866667,...,0,0,0,0,0,0,0,0,0,0
1,0.792157,0.784314,0.764706,0.745098,0.733333,0.737255,0.752941,0.752941,0.733333,0.737255,...,0,0,0,0,0,0,0,0,0,0
2,0.862745,0.866667,0.878431,0.886275,0.874510,0.862745,0.862745,0.866667,0.874510,0.878431,...,0,0,0,0,0,0,0,0,0,0
3,0.764706,0.764706,0.768627,0.764706,0.745098,0.737255,0.725490,0.729412,0.745098,0.760784,...,0,0,0,0,0,0,0,0,0,0
4,0.819608,0.796078,0.780392,0.776471,0.768627,0.768627,0.756863,0.768627,0.772549,0.756863,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20623,1.000000,1.000000,1.000000,0.996078,0.996078,0.996078,0.988235,0.992157,0.992157,0.996078,...,0,0,0,0,0,0,0,0,0,0
20624,0.949020,0.956863,0.952941,0.937255,0.937255,0.941176,0.956863,0.964706,0.956863,0.960784,...,0,0,0,0,0,0,0,0,0,0
20625,0.909804,0.909804,0.913725,0.917647,0.925490,0.925490,0.917647,0.913725,0.909804,0.913725,...,0,0,0,0,0,0,0,0,0,0
20626,0.941176,0.945098,0.949020,0.952941,0.952941,0.956863,0.956863,0.949020,0.941176,0.941176,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# Random Forest
rf_clf = RandomForestClassifier(n_estimators=100,random_state=22,max_depth=10,min_samples_split=5,max_leaf_nodes=100)
rf_clf.fit(X_train, y_train) 

y_pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy score with hog features concat to normalised df:", accuracy)

/tmp/ipykernel_20/1140014416.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf.fit(X_train, y_train)


Accuracy score with hog features concat to normalised df: 0.9386815317498788


In [31]:
# SVM Classifier
svm_clf = SVC(kernel='linear', random_state=42, probability=True)
svm_clf.fit(X_train, y_train)
y_pred = svm_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy score with hog features concat to normalised df (SVC):", accuracy)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy score with hog features concat to normalised df (SVC): 0.9580707707222491


In [32]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy score:", accuracy)

/opt/conda/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Accuracy score: 0.8928744546776539


# INFERENCE

In [33]:
class_folders = [folder for folder in os.listdir(test_path) if os.path.isdir(os.path.join(test_path, folder))]

classwise_imgs_paths = []
for image_dir in class_folders:
    p = [os.path.join(test_path, image_dir, filename) for filename in os.listdir(os.path.join(test_path, image_dir)) if filename.endswith(('.jpg', '.png'))]
    classwise_imgs_paths.append(p)

In [34]:
data_test = []
data_test_norm = []
data_edge_test = []
tru_labels = []
for c in classwise_imgs_paths:
    for image_p in c:
#         print(image_p)
        img = Image.open(image_p)
        img = img.convert('L')
        img = img.resize((38, 38), Image.Resampling.LANCZOS)
        img_arr = np.array(img).flatten()
        img_arr_norm = np.array(img).flatten() / 255.0
        edge_arr = compute_edges(np.array(img))
        edge_arr_pil = Image.fromarray(edge_arr)
        edge_arr_pil = edge_arr_pil.resize((38, 38), Image.Resampling.LANCZOS)
        edge_arr = np.array(edge_arr_pil).flatten()
        cl = image_p.split("/")[-2]
        tru_labels.append(cl)
        data_test.append(img_arr)
        data_test_norm.append(img_arr_norm)        
        data_edge_test.append(edge_arr)

In [35]:
data_df_test = pd.DataFrame(data_test, columns=pixel_cols)
data_df_norm_test = pd.DataFrame(data_test_norm, columns=pixel_cols)
test_df_edge = pd.DataFrame(data_edge_test, columns=edge_cols)
labels_df_test = pd.DataFrame(tru_labels)
data_df_test

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_1434,pixel_1435,pixel_1436,pixel_1437,pixel_1438,pixel_1439,pixel_1440,pixel_1441,pixel_1442,pixel_1443
0,186,185,182,181,184,183,184,185,182,180,...,143,133,128,127,129,131,140,156,173,178
1,227,225,226,224,220,220,221,219,223,224,...,220,212,202,195,192,193,202,215,221,223
2,215,210,206,202,200,202,204,202,206,204,...,161,158,156,160,159,158,162,174,194,204
3,245,244,241,238,235,235,236,238,241,245,...,245,241,235,220,201,192,197,217,235,240
4,199,199,198,197,197,196,195,197,197,196,...,196,193,182,170,158,149,151,160,176,185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,199,200,202,202,200,196,194,194,195,195,...,202,202,204,205,203,200,199,202,205,206
1004,247,247,247,247,247,247,247,248,249,249,...,233,241,245,245,245,246,248,248,248,248
1005,206,206,206,205,205,205,205,202,199,196,...,210,210,209,209,211,211,210,207,205,204
1006,172,171,170,168,169,171,169,169,171,172,...,164,165,167,169,170,169,168,171,170,169


In [36]:
test_df_edge = test_df_edge.replace({True: 1, False: 0})
test_df_edge

,edge_0,edge_1,edge_2,edge_3,edge_4,edge_5,edge_6,edge_7,edge_8,edge_9,...,edge_1434,edge_1435,edge_1436,edge_1437,edge_1438,edge_1439,edge_1440,edge_1441,edge_1442,edge_1443
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
hog_features_test = np.array([extract_hog_features(image.reshape(38, 38)) for image in data_df_test.values])
# lbp_features_test = np.array([extract_lbp_features(image.reshape(38, 38)) for image in data_df_test.values])
hog_features_df_test = pd.DataFrame(hog_features_test, columns=hog_cols)
# lbp_features_df_test = pd.DataFrame(lbp_features_test, columns=lbp_cols)

test_df_norm_w_hog_features_and_lbp = pd.concat([data_df_norm_test, hog_features_df_test, test_df_edge], axis=1)

In [38]:
test_predictions = model.predict(test_df_norm_w_hog_features_and_lbp.values)
answers = np.argmax(test_predictions, axis=1)
mapped_labels_df = pd.DataFrame()
mapped_labels_df["label"] = labels_df_test[0].map(alpha_map)

In [39]:
correct = 0
tot = 0
for i in range(len(answers)):
    if answers[i] == mapped_labels_df["label"].values[i]:
        correct +=1
    tot +=1
print(f"Accuracy = {(correct/tot)*100}%")

Accuracy = 98.90873015873017%


In [40]:
y_pred_rnd_for = rf_clf.predict(test_df_norm_w_hog_features_and_lbp)

accuracy_forest = accuracy_score(mapped_labels_df["label"].values, y_pred_rnd_for)
print("Accuracy on testing data:", accuracy_forest)

Accuracy on testing data: 0.9712301587301587


In [41]:
y_pred_svm = svm_clf.predict(test_df_norm_w_hog_features_and_lbp)

accuracy_svm = accuracy_score(mapped_labels_df["label"].values, y_pred_svm)
print("Accuracy on testing data:", accuracy_svm)

Accuracy on testing data: 0.9970238095238095


In [42]:
y_pred_knn = knn.predict(test_df_norm_w_hog_features_and_lbp)

accuracy_knn = accuracy_score(mapped_labels_df["label"].values, y_pred_knn)
print("Accuracy on testing data:", accuracy_knn)

Accuracy on testing data: 0.9751984126984127


## Trying out stacking

In [43]:
# svm_predictions = svm_clf.predict_proba(train_df_norm_w_hog_features_and_lbp)  # X_train contains your input data
# rf_predictions = rf_clf.predict_proba(train_df_norm_w_hog_features_and_lbp)
# knn_predictions = knn.predict_proba(train_df_norm_w_hog_features_and_lbp)
# mclr_predictions = model.predict(X)
svm_predictions = svm_clf.predict_proba(train_df_norm_w_hog_features_and_lbp)  # X_train contains your input data
rf_predictions = rf_clf.predict_proba(train_df_norm_w_hog_features_and_lbp)
knn_predictions = knn.predict_proba(train_df_norm_w_hog_features_and_lbp)
mclr_predictions = model.predict(train_df_norm_w_hog_features_and_lbp)

stacking_data = np.column_stack((svm_predictions, rf_predictions, knn_predictions, mclr_predictions))

In [44]:
# final_pred = (svm_predictions+rf_predictions+knn_predictions+mclr_predictions)/4.0
# answers_preds = np.argmax(final_pred, axis=1)
# correct = 0
# tot = 0
# for i in range(len(answers_preds)):
#     if answers_preds[i] == mapped_labels_df["label"].values[i]:
#         correct +=1
#     tot +=1
# print(f"Accuracy = {(correct/tot)*100}%")

In [45]:
# final_pred.shape

In [46]:
meta_model = SVC(kernel='linear', random_state=42, probability=True)
meta_model.fit(stacking_data, labels_df)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='linear', probability=True, random_state=42)

In [47]:
test_svm_predictions = svm_clf.predict_proba(test_df_norm_w_hog_features_and_lbp)
test_rf_predictions = rf_clf.predict_proba(test_df_norm_w_hog_features_and_lbp)
test_knn_predictions = knn.predict_proba(test_df_norm_w_hog_features_and_lbp)
test_mclr_predictions = model.predict(test_df_norm_w_hog_features_and_lbp)

In [48]:
stacking_data_test = np.column_stack((test_svm_predictions, test_rf_predictions, test_knn_predictions, test_mclr_predictions))
final_predictions = meta_model.predict(stacking_data_test)

In [49]:
final_predictions

array([23, 23, 23, ..., 19, 19, 19])

In [50]:
correct = 0
tot = 0
for i in range(len(final_predictions)):
    if final_predictions[i] == mapped_labels_df["label"].values[i]:
        correct +=1
    tot +=1
print(f"Accuracy = {(correct/tot)*100}%")

Accuracy = 99.60317460317461%
